In [166]:
import pandas as pd

In [165]:
pd.read_csv('https://gist.githubusercontent.com/slotix/11f0930b2d46d2946249a10e6216735b/raw/9ece7417bb5d13c032f1e070ace7142a908b133d/css3vsxpath.csv')

,Goal,CSS 3,XPath
0,All Elements,*,//*
1,All P Elements,p,//p
2,All Child Elements,p>*,//p/*
3,Element By ID,#foo,//*[@id=’foo’]
4,Element By Class,.foo,"//*[contains(@class,’foo’)]"
5,Element With Attribute,*[title],//*[@title]
6,First Child of All P,p>*:first-child,//p/*[0]
7,All P with an A child,Not possible,//p[a]
8,Next Element,p + *,//p/following-sibling::*[0]
9,Previous Element,Not possible,//p/preceding-sibling::*[0]


# Карты GP

Примерные точки карты, которые нужно получить. На них есть возможность перехода по другим 20 объектам

In [43]:
urls = ['https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@51.7125234,103.9733562,9z/data=!3m1!4b1!4m2!2m1!6e1',
       'https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@52.3985396,105.8933786,8z/data=!3m1!4b1!4m2!2m1!6e1',
       'https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@53.2908288,106.6973561,8z/data=!3m1!4b1!4m2!2m1!6e1',
       'https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@54.6275538,108.2470177,8z/data=!4m2!2m1!6e1',
       'https://www.google.ru/maps/search/%D0%B4%D0%BE%D1%81%D1%82%D0%BE%D0%BF%D1%80%D0%B8%D0%BC%D0%B5%D1%87%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8/@53.5897138,103.5288556,6.57z/data=!4m2!2m1!6e1']

In [44]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import time
import re
import json

In [45]:
driver = webdriver.Chrome()

In [125]:
driver.get(urls[-1])

![image7013d3a57ec52198.png](https://imgbb.su/images/2023/02/09/image7013d3a57ec52198.png)

## Автоскрол 

Найти область прокрутки и элементы блоков

In [135]:
area = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
elements = driver.find_elements(By.XPATH, '//div[@jstcache = "176"]') 
len(elements)

18

In [146]:
driver.execute_script("arguments[0].scrollTo(0, arguments[0].scrollHeight)", area)

In [147]:
height = driver.execute_script("return arguments[0].scrollHeight", area)
height

6684

In [148]:
elements = driver.find_elements(By.XPATH, '//div[@jstcache = "176"]') 

In [149]:
len(elements)

40

## Нажать на кнопку 
"К началу"

In [160]:
driver.execute_script("arguments[0].click();", driver.find_element(By.XPATH, '//button[@aria-label="К началу"]'))

## Поточный сбор

In [37]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from multiprocessing.dummy import Pool
from multiprocessing import cpu_count, freeze_support
import time
import re
import tqdm
import pandas as pd

In [23]:
for i in tqdm.tqdm_notebook(range(3)):
    time.sleep(0.2)

<ipython-input-23-34cbbdedfa6f>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm.tqdm_notebook(range(3)):


In [32]:
start_url = 'https://mirturbaz.ru'

def get_tables(values):
    driver = webdriver.Chrome()
    
#   Линия прогресса
    for url, reviews_count in tqdm.notebook.tqdm(values, total = len(values)):
        
        driver.get(start_url+url)
#       Ожидание прогрузки, делаем вид, что это быстрый человек и медленная программа  
        time.sleep(3)
        
#       Категория места
        text = ''
        for element in driver.find_elements(By.XPATH,"/html/body/div/div/script"):
            text = driver.execute_script("return arguments[0].innerHTML;", element)
            if 'window' in text:
                try:
                    category = re.search(r"window\.camp\.category = \'.*\'", text)[0].split("window.camp.category = '")[1][:-1]
                except:
                    category = ''
                break

#       Количество фото
        count_photo = ''
        try:
            count_photo = re.search(r'\d+(\s\d+)*', driver.find_element(By.XPATH, '//button[@rel="photo-popup"]').text)[0]
        except:
            pass

#       Описание услуг
        for service in driver.find_elements(By.XPATH,'//li[@class = "base-services-list-item base-services-box"]'):
            try:
                main = service.find_element(By.TAG_NAME,'h3').text

                h4 = []
                p = []
                for i in service.find_element(By.XPATH,'div[@class="base-services-box-info"]').find_elements(By.XPATH, 'div[@class="base-services-box-row"]'):
                    try:
                        h4.append(i.find_element(By.TAG_NAME, 'h4').text)
                    except:
                        h4.append('')
                    try:
                        p.append(i.find_element(By.TAG_NAME, 'p').text)
                    except:
                        p.append('')

                append_elements = list(set([i.text for i in service.find_elements(By.TAG_NAME, 'h4') if i.text !=''])-set(h4))

                h4.extend(append_elements)
                p.extend(['']*len(append_elements))

                service_items.append({'url': url,
                                      'service':main,
                                      'service_detail': h4,
                                      'description':p})
            except:
                continue

#       Как добраться
        routes = ''
        try:
            routes = driver.find_element(By.ID, 'routes').text
        except:
            pass
        
#       Популярные услуги
        has_pop_service = False
        try:
            for element in driver.find_elements(By.XPATH, '//span[contains(@class, "card__labels-item card__labels-item_recomend")]'):
                service_popular.append({'url': url,'service_popular':element.text})
            has_pop_service = True
        except:
            pass
#       Номерной фонд
        has_rooms = False
        try:
            for element in driver.find_elements(By.XPATH, '//li[@class= "rooms-list-item rooms-box"]'):
                el = element.find_element(By.CLASS_NAME, 'rooms-box-room')
                h3 =  el.find_element(By.TAG_NAME, 'h3').text
                room_description = ''
                count_place = ''
                other = ''
                area_room = ''
                for i in el.find_elements(By.TAG_NAME, 'li'):        
                    room_description+='- '+i.text
                    if 'местный' in i.text:
                        count_place = i.text
                    elif 'м\n2' in i.text:
                        area_room = re.search(r'\d+', i.text)[0]
                    else:
                        other = i.text

                price = ''
                try:
                    price = element.find_element(By.CLASS_NAME, 'rub-icon_black').text
                except:
                    pass

                rooms.append({'url':url,
                              'room_name':h3,
                              'full_description': room_description,
                              'count_place': count_place,
                              'room_area': area_room,
                              'other': other,
                              'price':price})
                has_rooms = True
        except:
            pass
        
#       Открыть все отзывы
        diff = reviews_count
        while diff != 0:
            try:
                review_items = driver.find_elements(By.CLASS_NAME, 'review-list-item')
                driver.execute_script("arguments[0].click();", driver.find_element(By.CLASS_NAME, 'review-list-btn'))
                time.sleep(3)
                
                if diff != reviews_count-len(review_items):
                    diff = reviews_count-len(review_items)
                else:
                    break
            except:
                break
#       Отзывы
        has_reviews = 0
        try:

            for review in review_items:
                author = review.find_element(By.CLASS_NAME, 'user-info-name').text
                rating = review.find_element(By.CLASS_NAME, 'card__rating-numb').text
                text = '\n\n'.join([i.text for i in review.find_elements(By.CLASS_NAME, "review-list-item__text")])
                date = review.find_element(By.CLASS_NAME, 'review-list-item__date').text
                type_trip = review.find_element(By.CLASS_NAME, 'review-list-item__type').text
                reviews.append({'url':url,
                                'author_name': author,
                                'date':date,
                                'rating': rating,
                               'text': text,
                               'type': type_trip})
            has_reviews = len(review_items)
            
            d = {'url':url}
            for i in driver.find_elements(By.CLASS_NAME, 'rating-numbers-item'):
                d[i.text.split(' ')[0]] = i.text.split(' ')[1][:-3] 
            ratings.append(d)
            
        except:
            pass

        items.append({'url':url,
                      'category': category,
                      'count_photo': count_photo,
                      'routs': routes,
                      'reviews_count_fact': has_reviews,
                      'has_rooms': has_rooms,
                      'has_pop_service': has_pop_service})

In [33]:
items = []
service_items = []
service_popular = []
rooms = []
reviews = []
ratings = []

In [34]:
urls = ['/russia/irkutskaya/azatey',
 '/russia/irkutskaya/usadba-daryana',
 '/russia/irkutskaya/panorama1',
 '/russia/irkutskaya/fort-baykal',
 '/russia/irkutskaya/domik-u-ozera',
 '/russia/irkutskaya/sharyjalgay',
 '/russia/irkutskaya/hadarta',
 '/russia/irkutskaya/baykal-yeti',
 '/russia/irkutskaya/baykal-mayami',
 '/russia/irkutskaya/baikalski']

reviews_count = [4, 4, 12, 1, 1, 2, 2, 3, 10, 3]


In [39]:
datalist = list(zip(urls, reviews_count))

if __name__== "__main__":
    freeze_support()
    pool = Pool()
    ITERATION_COUNT = cpu_count()-1 if len(datalist) >= cpu_count()-1 else len(datalist)

    count_per_iteration = len(datalist) / float(ITERATION_COUNT)

    for i in range(0, ITERATION_COUNT):
        list_start = int(count_per_iteration * i)
        list_end = int(count_per_iteration * (i+1))
        pool.apply_async(get_tables, [datalist[list_start:list_end]])

In [40]:
df_items = pd.DataFrame(items)
df_rooms = pd.DataFrame(rooms)
df_services = pd.DataFrame(service_items).explode(['service_detail', 'description'])
df_services_p = pd.DataFrame(service_popular)
df_reviews = pd.DataFrame(reviews)
df_ratings = pd.DataFrame(ratings)

In [41]:
df_items

,url,category,count_photo,routs,reviews_count_fact,has_rooms,has_pop_service
0,/russia/irkutskaya/sharyjalgay,База отдыха,39,Как добраться,2,True,True
1,/russia/irkutskaya/domik-u-ozera,База отдыха,36,Как добраться,1,True,True
2,/russia/irkutskaya/baykal-yeti,Гостевой дом,45,Как добраться\nАвтотранспорт.\nИз Иркутска по ...,3,True,True
3,/russia/irkutskaya/hadarta,База отдыха,45,Как добраться\nЛетом из Иркутска от стадиона «...,2,True,True
4,/russia/irkutskaya/usadba-daryana,Гостевой дом,35,Как добраться\nТрансфер\nДоставка осуществляет...,4,True,True
5,/russia/irkutskaya/baykal-mayami,База отдыха,35,Как добраться\nБаза отдыха находится в 250 мет...,10,True,True
6,/russia/irkutskaya/panorama1,Гостевой дом,26,Как добраться\nВариант 1. Самостоятельно добра...,12,True,True
7,/russia/irkutskaya/azatey,Гостиный двор,32,Как добраться\nМожно доехать на собственном ав...,4,True,True
8,/russia/irkutskaya/baikalski,Гостиница,25,Как добраться,3,True,True
9,/russia/irkutskaya/fort-baykal,Туристическая база,45,Как добраться\nСпособы проезда до турбазы «Фор...,1,True,True


In [42]:
df_rooms

,url,room_name,full_description,count_place,room_area,other,price
0,/russia/irkutskaya/sharyjalgay,Коттедж № 5 (2-й этаж над сауной),- 7-и местный дом,7-и местный дом,,,от 3300 ₽
1,/russia/irkutskaya/sharyjalgay,Коттедж № 3/4,- 8-и местный дом,8-и местный дом,,,от 3300 ₽
2,/russia/irkutskaya/sharyjalgay,Коттедж № 1/2,- 9-и местный дом,9-и местный дом,,,от 3300 ₽
3,/russia/irkutskaya/sharyjalgay,Эконом,- 2-х местный номер,2-х местный номер,,,от 300 ₽
4,/russia/irkutskaya/sharyjalgay,Стандарт,- 2-х местный номер,2-х местный номер,,,от 500 ₽
5,/russia/irkutskaya/domik-u-ozera,Мини гостиница,- 2-х местный номер,2-х местный номер,,,от 5000 ₽
6,/russia/irkutskaya/domik-u-ozera,Дом,- 5-и местный дом- 60м\n2,5-и местный дом,60,,от 5000 ₽
7,/russia/irkutskaya/domik-u-ozera,Новый дом,- 10-и местный дом- 70м\n2,10-и местный дом,70,,от 8889 ₽
8,/russia/irkutskaya/domik-u-ozera,Второй новый дом,- 10-и местный дом,10-и местный дом,,,от 24000 ₽
9,/russia/irkutskaya/baykal-yeti,Логово Йети,- 2-х местный номер,2-х местный номер,,,от 3000 ₽
